# Retriever Reader Pipeline

We've setup our Elasticsearch instance and are now ready to prepare the remainder of our ODQA pipeline - our *retriever* and *reader*.

First we initialize the connection to our Elasticsearch document store.

In [3]:
# from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

# doc_store = ElasticsearchDocumentStore(
#     host='localhost',
#     username='', password='',
#     index='aurelius'
# )

def reading(file_name = '../11_reader-retriever_qa_with_haystack/credentials.txt'):
    s = open(file_name, 'r').read()
    dict = eval(s)
    return(dict)

credential_dict = reading()

from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    scheme='https', 
    username=credential_dict['username'], 
    password=credential_dict['pwd'], 
    ca_certs=credential_dict['ca_certs'],
    index='aurelius'
)

Then we initialize our retriever and reader models. We will be using [deepset/bert-base-cased-squad2](https://huggingface.co/deepset/bert-base-cased-squad2) as our reader model.

In [6]:
# from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.nodes import BM25Retriever


# intialize BM25 retriever
# retriever = ElasticsearchRetriever(doc_store)  # BM25
retriever = BM25Retriever(doc_store)

In [7]:
# from haystack.reader.farm import FARMReader
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

And now we initialize our ODQA pipeline.

In [8]:
# from haystack.pipeline import ExtractiveQAPipeline
from haystack.pipelines import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

Now we can begin asking questions!

In [9]:
qa.run(query='What did your grandfather teach?')

Inferencing Samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.95s/ Batches]


{'query': 'What did your grandfather teach?',
 'no_ans_gap': -3.121255397796631,
 'answers': [<Answer {'answer': 'good morals and the government of my temper', 'type': 'extractive', 'score': 0.16657505929470062, 'context': 'From my grandfather Verus I learned good morals and the government of my temper.', 'offsets_in_document': [{'start': 36, 'end': 79}], 'offsets_in_context': [{'start': 36, 'end': 79}], 'document_id': 'ecc466cb76ed8dedab8752cc84e411b4', 'meta': {'source': 'meditations'}}>,
  <Answer {'answer': 'thy life under thy mother', 'type': 'extractive', 'score': 0.024128783494234085, 'context': 'Termination of activity, cessation from movement and opinion, and in a sense their death, is no evil. Turn thy thoughts now to the consideration of thy life, thy life as a child, as a youth, thy manhood, thy old age, for in these also every change was a death. Is this anything to fear? Turn thy thoughts now to thy life under thy grandfather, then to thy life under thy mother, then to th

In [14]:
# qa.run(query='What is the Universe?', top_k_reader=3)
qa.run(query='What is the Universe?', params={"Reader": {"top_k": 3}})

Inferencing Samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.81 Batches/s]


{'query': 'What is the Universe?',
 'no_ans_gap': 9.206116199493408,
 'answers': [<Answer {'answer': 'the universe loves to make whatever is about to be', 'type': 'extractive', 'score': 0.7264758348464966, 'context': '"The earth loves the shower"; and "the solemn aether loves": and the universe loves to make whatever is about to be. I say then to the universe, that I love as thou lovest. And is not this too said, that "this or that loves (is wont) to be produced"?', 'offsets_in_document': [{'start': 65, 'end': 115}], 'offsets_in_context': [{'start': 65, 'end': 115}], 'document_id': 'dfe112a37a09939594906ff8a0d7be8a', 'meta': {'source': 'meditations'}}>,
  <Answer {'answer': 'a well-arranged universe', 'type': 'extractive', 'score': 0.7179001569747925, 'context': 'Either it is a well-arranged universe or a chaos huddled together, but still a universe. But can a certain order subsist in thee, and disorder in the All? And this too when all things are so separated and diffused and sympathe

In [16]:
# qa.run(query='What is art?', top_k_reader=3)
qa.run(query='What is arte?', params={"Reader": {"top_k": 3}})

Inferencing Samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.92 Batches/s]


{'query': 'What is arte?',
 'no_ans_gap': 3.995941162109375,
 'answers': [<Answer {'answer': 'the body', 'type': 'extractive', 'score': 0.7202165126800537, 'context': 'Turn it (the body) inside out, and see what kind of thing it is; and when it has grown old, what kind of thing it becomes, and when it is diseased.', 'offsets_in_document': [{'start': 9, 'end': 17}], 'offsets_in_context': [{'start': 9, 'end': 17}], 'document_id': 'edf0d5c6c04f02a0359cfc4f90699cea', 'meta': {'source': 'meditations'}}>,
  <Answer {'answer': 'it is useful to act according to nature', 'type': 'extractive', 'score': 0.5466973781585693, 'context': 'No man is tired of receiving what is useful. But it is useful to act according to nature. Do not then be tired of receiving what is useful by doing it to others.', 'offsets_in_document': [{'start': 49, 'end': 88}], 'offsets_in_context': [{'start': 49, 'end': 88}], 'document_id': '3cce136d95420cdb2ae7ae7933dbaf9f', 'meta': {'source': 'meditations'}}>,
  <Answer {'ans